<a href="https://colab.research.google.com/github/Forpee/NLP-practice/blob/master/multiclass_classification_exercise/draft_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Freestyle


#Imports

In [1]:
import tensorflow as tf
import os

from tensorflow.keras import layers


##Check for GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-86216e12-3186-76b9-9f42-f25af531bf32)


#download dataset

In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

path = tf.keras.utils.get_file(
    fname='stackoverflow_questions',
    origin=data_url,
    untar=True
)


In [4]:
dirname=os.path.dirname(path)
dirname

'/root/.keras/datasets'

In [5]:
os.listdir(dirname)

['test', 'README.md', 'train', 'stackoverflow_questions.tar.gz']

In [6]:
train_dirname=os.path.join(dirname,'train')
train_dirname

'/root/.keras/datasets/train'

In [7]:
test_dirname=os.path.join(dirname,'test')
test_dirname

'/root/.keras/datasets/test'

#create dataset

In [8]:
seed=42

train_ds=tf.keras.utils.text_dataset_from_directory(train_dirname,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [9]:
val_ds=tf.keras.utils.text_dataset_from_directory(train_dirname,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [10]:
test_ds=tf.keras.utils.text_dataset_from_directory(test_dirname)

Found 8000 files belonging to 4 classes.


#encoder

In [11]:
text_vectorizer=tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=250
)

train_text=train_ds.map(lambda text, labels: text)

text_vectorizer.adapt(train_text)


#perfomance

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

#embeddings


In [13]:
embedding=layers.Embedding(input_dim=10001, output_dim=64)

##sequential model first

In [29]:
num_classes=4

seq_model=tf.keras.Sequential([
    text_vectorizer,
    embedding,
  layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
    # layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

#compile model

In [30]:
seq_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=['accuracy'])

In [31]:
history_0=seq_model.fit(train_ds,
                    epochs=10,
                    validation_data=val_ds)

Epoch 1/10
200/200 [==============================] - 9s 8ms/step - loss: 0.7646 - accuracy: 0.6955 - val_loss: 0.6364 - val_accuracy: 0.7337
Epoch 2/10
200/200 [==============================] - 2s 8ms/step - loss: 0.4953 - accuracy: 0.8080 - val_loss: 0.5710 - val_accuracy: 0.7763
Epoch 3/10
200/200 [==============================] - 1s 7ms/step - loss: 0.3572 - accuracy: 0.8722 - val_loss: 0.5421 - val_accuracy: 0.7950
Epoch 4/10
200/200 [==============================] - 1s 7ms/step - loss: 0.2520 - accuracy: 0.9230 - val_loss: 0.5395 - val_accuracy: 0.8019
Epoch 5/10
200/200 [==============================] - 1s 7ms/step - loss: 0.1688 - accuracy: 0.9580 - val_loss: 0.5549 - val_accuracy: 0.8006
Epoch 6/10
200/200 [==============================] - 1s 7ms/step - loss: 0.1060 - accuracy: 0.9791 - val_loss: 0.5809 - val_accuracy: 0.8012
Epoch 7/10
200/200 [==============================] - 2s 8ms/step - loss: 0.0629 - accuracy: 0.9909 - val_loss: 0.6127 - val_accuracy: 0.8000
Epoch 